# 4 Feature Engineering
**Prerequisites:** Preprocessing is finalised. Proc file has been created.

**Objective:** Build the model features LRFMP.


## Library Imports

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pandas_profiling
from functools import reduce

## 4.1 Import Data

masterdata of the customers & processed transaction data

In [2]:
df_master = pd.read_csv("../01_data preprocessing/data new/masterdata.csv",encoding="utf-8",sep=",",
                        parse_dates=["ValidityDate"],usecols=[0,1,11],dtype={"CardID":object})
df_master1 = df_master.sort_values(["CustID","ValidityDate"]).loc[~df_master.duplicated(subset=["CustID"],keep="first")]
df_master1 = df_master1.loc[:,["CustID","ValidityDate"]]

In [3]:
df_proc = pd.read_csv("../01_data preprocessing/data new/procdata.csv",encoding="utf-8",sep=",",
                      parse_dates=["BookingDate"],dtype={"CustID":"int64"})
df_proc["Counter"] = df_proc.groupby("CustID").cumcount(ascending=False) + 1
df_proc1 = df_proc.loc[:,["CustID","CardID","Store","BookingDate","Counter"]]

Inserting a customer sample to analyse the following steps.

In [4]:
#define a observation customer
customer = 1604011635399

In [5]:
df_proc.loc[df_proc["CustID"].isin([customer])]

,CustID,CardID,BookingDate,Store,TransactionType,Revenue,RevenueInc.,Qty,Price/Article,Counter
29,1604011635399,55827754899,2017-05-28,9,3,28.54,28.54,1.0,28.54,8
30,1604011635399,55827754899,2017-08-24,1,1,39.42,39.42,1.0,39.42,7
31,1604011635399,55827754899,2017-08-25,8,1,41.88,41.88,1.0,41.88,6
32,1604011635399,55827754899,2017-10-10,1,1,127.85,127.85,7.0,18.26,5
33,1604011635399,55827754899,2017-11-30,6,1,46.50,45.50,1.0,46.50,4
34,1604011635399,55827754899,2018-03-10,10,1,25.91,25.91,1.0,25.91,3
35,1604011635399,55827754899,2018-04-16,5,1,25.91,25.91,1.0,25.91,2
36,1604011635399,55827754899,2018-07-18,2,1,24.16,24.16,1.0,24.16,1


## 4.2 Feature Engineering

### 4.2.1 Length feature
**1. Version:** last purchase - first purchase    
**2. Version:** last purchase - validity date

**Method:** Firstly, the processed transaction data has to be enriched by the validity date. Secondly, building of first & last purchase per customer by using a copy of booking date and a groupby by customer (first = min, last = max).

**Problem:** *Customers with multiple cards!* In this case the validity date of the oldest card will be used.

**Selected for clustering:**   last purchase - validity date

In [6]:
df_l = pd.merge(df_proc,df_master1,how="left",on="CustID",indicator=True)
df_l["BD_copy"] = df_l["BookingDate"]
df_l1 = df_l.loc[:,["CustID","Store","BookingDate","BD_copy","ValidityDate"]]
df_l1.rename(columns={"BookingDate": "LastPurchase","BD_copy": "FirstPurchase"},inplace=True)

In [7]:
df_l2 = df_l1.groupby(["CustID"], as_index=False).agg({"LastPurchase": np.max,
                                                       "FirstPurchase": np.min,
                                                       "ValidityDate": np.min})
    
df_l2["Length (Validity)"] = (df_l2["LastPurchase"] - df_l2["ValidityDate"]).dt.days
df_l2["Length (First/Last)"] = (df_l2["LastPurchase"] - df_l2["FirstPurchase"]).dt.days

In [8]:
#Check via customer sample
df_l2.loc[df_l2["CustID"].isin([customer])]

,CustID,LastPurchase,FirstPurchase,ValidityDate,Length (Validity),Length (First/Last)
1,1604011635399,2018-07-18,2017-05-28,2016-02-12,887,416


### 4.2.2 Recency feature

**1. Version:** Recency(1) as the days since the last purchase    
**2. Version:** Recency(3) as the average days between the last 3 purchases


**Method:** Indexing of the columns and creating of a counter variable for the purchases, selecting the customers first three purchases, calculating the timedifference, inserting r(1) and r(2), grouping and calculating the features.


**Selected for clustering:** Recency(3) as the average days between the last 3 purchases

In [9]:
customer_purchases_1to3 = df_proc1["Counter"].isin(["1","2","3"])
df_r = df_proc1.loc[customer_purchases_1to3,:].copy()

In [10]:
#define observation period
end_of_period = pd.Timestamp("2018-07-30")

datediff = lambda x: (end_of_period - x)

df_r["Recency (1)"] = (df_r.groupby("CustID")["BookingDate"].transform(datediff)).dt.days
df_r["Recency (3)"] = df_r["Recency (1)"]

In [11]:
df_r1 = df_r.groupby(["CustID"], as_index=False).agg({"Recency (1)": np.min, 
                                                      "Recency (3)": np.mean,
                                                      "Counter":np.max})

df_r1["Recency (3)"] = np.round(df_r1["Recency (3)"], decimals = 2)

In [12]:
df_r1.loc[~df_r1["Counter"].isin(["3"])].to_csv("data excluded/excluded 7_customers without less than 3 purchases.csv",
                                       encoding="utf-8", sep=",")

print("Customers with less than 3 purchases: " + str(len(df_r1.loc[~df_r1["Counter"].isin(["3"])])))

Customers with less than 3 purchases: 0


In [13]:
customer_with_purchases_3ormore = df_r1["Counter"].isin(["3"])
df_r2 = df_r1.loc[customer_with_purchases_3ormore,:]

In [14]:
#Check via customer sample
df_r2.loc[df_r2["CustID"].isin([customer])]

,CustID,Recency (1),Recency (3),Counter
1,1604011635399,12,86.33,3


### 4.2.3 Frequency feature
**1. Version:** Frequency as the number of purchases in the observation period    
**2. Version:** Frequency as the average number of purchases per month since the customers first purchase


**Method:** Creating a copy of the booking date, indexing the columns and inserting a counter variable, group the dataframe and calculate the column values, adding a column for month in order to calculate the factual program memberhsip, offsetting last purchase to receive the right result, calculate the features and rename the columns.

**Selected for Clustering:** Frequency as the average number of purchases per month since the customers first purchase

In [15]:
df_f = pd.merge(df_proc1, df_master1, how="left", on="CustID", indicator=True)
df_f["BD_copy"] = df_f["BookingDate"]
df_f1 = df_f.loc[:,["CustID","Store","BookingDate","BD_copy"]]
df_f1["Counter"] = df_f1.groupby("CustID").cumcount(ascending=False) + 1

df_f1.rename(columns={"BookingDate": "LastPurchase", "BD_copy": "FirstPurchase"}, inplace=True)

In [16]:
df_f2= df_f1.groupby(["CustID"], as_index=False).agg({"LastPurchase": np.max,
                                                      "FirstPurchase": np.min,
                                                      "Counter": np.max})

df_f2["LastPurchase(M)"] = df_f2["LastPurchase"].dt.to_period("M").dt.to_timestamp()
df_f2["FirstPurchase(M)"] = df_f2["FirstPurchase"].dt.to_period("M").dt.to_timestamp()

now = max(df_f2["LastPurchase(M)"])+ pd.offsets.MonthBegin(2)
df_f2["Offset(M)"] = now

df_f2["Validity(M)"] = (df_f2["Offset(M)"] - df_f2["FirstPurchase(M)"]).astype("timedelta64[M]")

df_f2["Frequency (Monthly)"] = np.round(df_f2["Counter"].divide(df_f2["Validity(M)"]), decimals=2)

df_f2.rename(columns={"Counter": "Frequency"}, inplace=True)

In [17]:
#Check via customer sample
df_f2.loc[df_f2["CustID"].isin([customer])]

,CustID,LastPurchase,FirstPurchase,Frequency,LastPurchase(M),FirstPurchase(M),Offset(M),Validity(M),Frequency (Monthly)
1,1604011635399,2018-07-18,2017-05-28,8,2018-07-01,2017-05-01,2018-09-01,16.0,0.5


### 4.2.4 Monetary features

**1. Version:** Revenue (total)    
**2. Version:** Revenue/Purchase    
**3. Version:** Price/Article    
**4. Version:** Qty/Purchase

**Selected for Clustering:** Revenue (total) and Revenue/Purchase

In [18]:
df_m = df_proc.groupby(["CustID"], as_index=False).agg({"Revenue": np.sum,
                                                        "Qty": np.sum,
                                                        "Counter": np.max})
                                                            
df_m["Revenue/Purchase"] = np.round(df_m["Revenue"].divide(df_m["Counter"]), decimals=2)
df_m["Price/Article"] = np.round(df_m["Revenue"].divide(df_m["Qty"]), decimals=2)
df_m["Revenue"] = np.round(df_m["Revenue"], decimals=2)
df_m["Qty/Purchase"] = np.round(df_m["Qty"].divide(df_m["Counter"]), decimals=2)

df_m.rename(columns={"Revenue": "Monetary (Revenue)", 
                     "Revenue/Purchase": "Monetary (Revenue/Purchase)",
                     "Counter": "PurchasesTotal"},inplace=True)

In [19]:
#Check via customer sample
df_m.loc[df_m["CustID"].isin([customer])]

,CustID,Monetary (Revenue),Qty,PurchasesTotal,Monetary (Revenue/Purchase),Price/Article,Qty/Purchase
1,1604011635399,360.17,14.0,8,45.02,25.73,1.75


### 4.2.5 Periodicity features

**1. Version:** Periodicity as the standard deviation of the intervall between two purchases


**Method:** Selecting the columns and adding of a count variable, define the engineering method (IVT = time between 2 puchases), adding periodicity columns via IVT, grouping of the dataframe and feature engineering

In [20]:
df_p = df_proc1.groupby(["CustID","BookingDate","Store","Counter"],as_index=False).agg(np.sum)
df_p1 = df_p.loc[:,["CustID","BookingDate","Store","Counter"]]

ivtdiff = lambda x: abs(x.shift(1) - x)

df_p1["IVT"] = (df_p1.groupby("CustID")["BookingDate"].transform(ivtdiff)).dt.days
df_p1["Periodicity"] = df_p1["IVT"]

In [21]:
df_p2 = df_p1.groupby(["CustID"], as_index=False).agg({"IVT": [np.max,np.min],
                                                       "Counter": [np.max],
                                                       "Periodicity": [np.mean,np.std]})

df_p2.columns = [" ".join(col).strip() for col in df_p2.columns.values]

df_p2["Periodicity (Mean)"] = np.round(df_p2["Periodicity mean"], decimals = 2)
df_p2["Periodicity (Std)"] = np.round(df_p2["Periodicity std"], decimals = 2)

In [22]:
customer_with_purchases_3ormore = df_p2["Counter amax"] >= 3
df_p3 = df_p2.loc[customer_with_purchases_3ormore]

In [23]:
#Check via customer sample
df_p3.loc[df_p3["CustID"].isin([customer])]

,CustID,IVT amax,IVT amin,Counter amax,Periodicity mean,Periodicity std,Periodicity (Mean),Periodicity (Std)
1,1604011635399,100.0,1.0,8,59.428571,35.957647,59.43,35.96


## 4.3 Merging Features
### 4.3.1 Preselection for clustering

#### Not used in clustering, because of redundancy or high correlation

- **Length:** LastPurchase, FirstPurchase, ValidityDate, Length (Validity)
- **Recency:** Recency(1), Counter
- **Frequency:** Frequency as the total number of purchases, LastPurchase(M), FirstPurchase, ValidityDate, Offset(M)
- **Monetary:** Purchases, Price/Articel, Qty/Purchase, Qty
- **Periodicity:** IVT max, IVT min, Counter amax, Periodicity (Mean)


#### Used in Clustering:

- **Length:** last purchase - validity date
- **Recency:** Recency(3) as the average days between the last 3 purchases
- **Frequency:** Frequency as the average number of purchases per month since the customers first purchase
- **Monetary:** Revenue (total) and Revenue/Purchase
- **Periodicity:** Periodicity as the standard deviation of the intervall between two purchases

In [24]:
df_cust = df_r2.loc[:,["CustID"]]

df_f2 = df_f2.loc[:,df_f2.columns != "LastPurchase"]
df_f2 = df_f2.loc[:,df_f2.columns != "FirstPurchase"]

In [25]:
dfs = [df_cust, df_l2, df_r2, df_f2, df_m, df_p3]
df_features = reduce(lambda left,right: pd.merge(left,right,how="left",on="CustID"), dfs)

In [26]:
#Export all features in a file to check correlations
df_features_checkcorr = df_features
df_features_checkcorr.to_csv("data new/features.csv", encoding="utf-8", sep=",", index=False)

In [27]:
df_features2= df_features.loc[:,["CustID","Length (First/Last)","Recency (3)","Frequency (Monthly)",
                                 "Monetary (Revenue)","Monetary (Revenue/Purchase)","Periodicity (Std)"]]

In [28]:
#Saving features in csv
df_features2.to_csv("data new/featureslrfmp.csv", encoding="utf-8", sep=",", index=False)

In [29]:
#Check via customer sample
df_features2.loc[df_features2["CustID"].isin([customer])]

,CustID,Length (First/Last),Recency (3),Frequency (Monthly),Monetary (Revenue),Monetary (Revenue/Purchase),Periodicity (Std)
1,1604011635399,416,86.33,0.5,360.17,45.02,35.96


### 4.3.2 Adding unutilized columns to variables

In [30]:
df_variables = pd.read_csv("../01_data preprocessing/data new/variables.csv",encoding="utf-8",sep=",")

In [31]:
df_variables = pd.merge(df_variables,df_features,how="left",on="CustID",indicator=True)

In [32]:
df_variables = df_variables.loc[~(df_variables["_merge"] == "left_only")]
df_variables = df_variables.loc[:,df_variables.columns != "_merge"]

print("Data entries variables: " + str(len(df_variables.index)))

Data entries variables: 70


In [33]:
#Saving variables
df_variables.to_csv("../01_data preprocessing/data new/variables.csv", encoding="utf-8", sep=",", index=False)

In [34]:
#Check via customer sample
df_variables.loc[df_variables["CustID"].isin([customer])]

,CustID,Revenue,OnlineRevenue,OfflineRevenue,ChannelShopper,ChannelShopperDetail,ChristmasRevenue,LastPurchase,FirstPurchase,ValidityDate,...,Monetary (Revenue/Purchase),Price/Article,Qty/Purchase,IVT amax,IVT amin,Counter amax,Periodicity mean,Periodicity std,Periodicity (Mean),Periodicity (Std)
1,1604011635399,360.17,167.27,192.9,Hybrid,HybridOn(>0.5),0.0,2018-07-18,2017-05-28,2016-02-12,...,45.02,25.73,1.75,100.0,1.0,8,59.428571,35.957647,59.43,35.96
